# Checking whether we're getting the same results

In [1]:
import pickle
import networkx as nx

In [2]:
with open('stroke_graph_anvy.pickle', 'rb') as handle:
    G_anvy = pickle.load(handle)
with open('stroke_graph_clse.pickle', 'rb') as handle:
    G_clse = pickle.load(handle)

Manually checking: edge attributes?

In [3]:
G_anvy.edges[(0,9)]

{'geometry': <LINESTRING (1603374.663 6464077.898, 1603342.343 6464406.368)>,
 'angles': [36.134980718680936]}

In [4]:
G_clse.edges[(0,9)]

{'geometry': <LINESTRING (1603374.663 6464077.898, 1603342.343 6464406.368)>,
 'number_connections': 1,
 'angles': [36.134980718680936]}

@csebastiao is the `number_connections` used for any of the metrics, or just a nice-to-have metric in itself?

In [5]:
# make sure we have the same edges
assert list(G_anvy.edges) == list(G_clse.edges), "Edges differ"

In [6]:
for edge in G_anvy.edges:
    assert G_anvy.edges[edge]["geometry"] == G_clse.edges[edge]["geometry"], "Geoms differ"
    print(f"Edge {edge}")
    angles_anvy = [round(angle, 10) for angle in sorted(G_anvy.edges[edge]["angles"])]
    angles_clse = [round(angle, 10) for angle in sorted(G_clse.edges[edge]["angles"])]
    if angles_anvy == angles_clse:
        print("Angles equal")
    else:
        print("Angles differ:")
        print(angles_anvy)
        print(angles_clse)
    print("\n")
    #assert sortedG_anvy.edges[edge]["angles"] == G_clse.edges[edge]["angles"], "Angles differ"

Edge (0, 2)
Angles equal


Edge (0, 9)
Angles equal


Edge (0, 3)
Angles differ:
[29.3960283634]
[27.9586406474]


Edge (0, 1)
Angles differ:
[89.7456019245, 89.7526780498]
[88.8925817012, 89.7526780498]


Edge (0, 4)
Angles differ:
[89.4291516617, 89.5662303351]
[63.1494214841, 63.2766055722]


Edge (1, 8)
Angles differ:
[70.0411369582]
[61.6127991329]


Edge (1, 3)
Angles differ:
[89.5308449728, 89.5858181738]
[83.0252283916, 89.5858181738]


Edge (1, 6)
Angles equal


Edge (2, 9)
Angles equal


Edge (2, 3)
Angles differ:
[34.2514380149]
[35.6888257308]


Edge (2, 6)
Angles differ:
[47.164433709, 59.7941982077, 80.1697662773, 84.2388688128]
[47.164433709, 59.7941982077, 59.7941982077, 88.6015042934]


Edge (3, 9)
Angles differ:
[88.08366042]
[89.5210481359]


Edge (3, 4)
Angles differ:
[88.7883380112, 89.197881055]
[71.9046849747, 74.1965981362]


Edge (4, 7)
Angles equal


Edge (4, 5)
Angles equal


Edge (4, 6)
Angles differ:
[86.2834611844, 88.6226495629]
[76.6579161543, 89.3460577

@csebastiao re the above. WHY :D 

Checking nodes

In [7]:
assert list(G_anvy.nodes) == list(G_clse.nodes), "Nodes differ"

In [8]:
G_anvy.nodes[0]

{'edge_indeces': [0, 3, 15, 27],
 'geometry': <POINT (1603374.663 6464077.898)>,
 'geometry_stroke': <LINESTRING (1603278.899 6463669.186, 1603283.731 6463690.028, 1603314.444 6...>,
 'x': 1603374.6625343116,
 'y': 6464077.898491419,
 'connectivity': 8,
 'degree': 5,
 'betweenness_centrality': 0.13657407407407404,
 'closeness_centrality': 0.6923076923076923,
 'connectivity_computed': 8,
 'access': 3,
 'length': 839.5666838320316,
 'spacing': 104.94583547900395,
 'orthogonality': 68.74678997354196}

@csebastiao: `connectivity_computed` can be ignored (we can later drop it entirely, was just a sanity check)

In [9]:
G_clse.nodes[0]

{'n_segments': 8,
 'geometry_stroke': <LINESTRING (1603278.899 6463669.186, 1603283.731 6463690.028, 1603314.444 6...>,
 'edge_ids': array([ 0,  3, 15, 27]),
 'geometry': <POINT (1603374.663 6464077.898)>,
 'x': array('d', [1603374.6625343116]),
 'y': array('d', [6464077.898491419]),
 'length': 839.5666838320316,
 'stroke_betweenness': 0.13657407407407404,
 'stroke_closeness': 0.6923076923076923,
 'stroke_degree': 5,
 'stroke_connectivity': 8,
 'stroke_access': 3,
 'stroke_orthogonality': 61.889319613560986,
 'stroke_spacing': 104.94583547900395}

In [10]:
# k:v is anvy:clse naming of node attrs
metrics_map = {
    "degree": "stroke_degree",
    "betweenness_centrality": "stroke_betweenness",
    "closeness_centrality": "stroke_closeness",
    "connectivity": "stroke_connectivity",
    "access": "stroke_access",
    "spacing": "stroke_spacing",
    #"orthogonality": "stroke_orthogonality"
}

In [11]:
for n in G_anvy.nodes:
    assert G_anvy.nodes[n]["x"] == G_clse.nodes[n]["x"][0], "x coords differ"
    assert G_anvy.nodes[n]["y"] == G_clse.nodes[n]["y"][0], "y coords differ"
    assert G_anvy.nodes[n]["geometry"] == G_clse.nodes[n]["geometry"], "geometries differ"
    assert G_anvy.nodes[n]["geometry_stroke"] == G_clse.nodes[n]["geometry_stroke"], "geometry_stroke differ"
    assert G_anvy.nodes[n]["edge_indeces"] == list(G_clse.nodes[n]["edge_ids"]), "Edge IDs differ"
    assert G_anvy.nodes[n]["length"] == G_clse.nodes[n]["length"]
    for k, v in metrics_map.items():
        assert round(G_anvy.nodes[n][k], 10) == round(G_clse.nodes[n][v], 10), f"{k} differ"
    

@csebastiao we get the same metrics everywhere but orthogonality (which i guess makes sense cause the angles are off?)

In [12]:
for n in G_anvy.nodes:
    ortho_anvy = round(G_anvy.nodes[n]["orthogonality"], 10)
    ortho_clse = round(G_clse.nodes[n]["stroke_orthogonality"], 10)
    if ortho_anvy == ortho_clse:
        print(f"{n}: same ortho")
    else:
        print(f"{n}: orthos differ")
        print(ortho_anvy)
        print(ortho_clse)    

0: orthos differ
68.7467899735
61.8893196136
1: orthos differ
86.3237109565
83.6925485182
2: orthos differ
60.6750720203
58.8531289111
3: orthos differ
72.6905727159
67.4115491701
4: orthos differ
87.2833822408
76.9079599519
5: same ortho
6: orthos differ
76.5085090591
72.9492134207
7: same ortho
8: orthos differ
78.9162659216
74.6909105965
9: orthos differ
59.3502878479
59.8294170866
